# Titanic Survivor Prediction

## 사용 모듈 불러오기

In [2]:
import numpy as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rc("font", family="AppleGothic")

## Train data와 Test data 불러오기

- Pclass: 선실 등급
- sibsp: 형제자매
- Parch: 부모, 자식
- Fare: 티켓 요금
- cabin: 선실
- Embarked: 승선지
##### info()와 describe() 사용하여 데이터 먼저 훑어보기

In [90]:
titanic_train = pd.read_csv("./titanic/titanic_train.csv", header=0, sep=",")
titanic_test = pd.read_csv("./titanic/titanic_test.csv", header=0, sep=",")

## 통계적 검증하기

In [194]:
# # 성별에 따른 생존 결과
# print(titanic_train.groupby(["Sex","Survived"])["Survived"].count())
# sns.barplot(x="Sex", y="Survived", data=titanic_train)

In [195]:
# # 선실 등급에 따른 생존 결과
# print(titanic_train.groupby(["Pclass","Survived"])["Survived"].count())
# sns.barplot(x="Pclass", y="Survived", data=titanic_train)

In [205]:
# # 나이 등급에 따른 생존 결과
# group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young adult', 'Adult', 'Elder']
# titanic_train["Age_Range"]=titanic_train["Age"].apply(lambda x: "Baby" if x<=5 else "Child" if x<=12 else "Teenager" if x<=18 else "Student" if x<=25 else "Young adult" if x<=35 else "Adult" if x<=60 else "Elderly" if x>=61 else "Unknown")
# plt.figure(figsize=(10,6))
# print(titanic_train.groupby(["Age_Range","Survived"])["Survived"].count())
# sns.barplot(x="Age_Range", y="Survived", hue="Sex", data=titanic_train, order=group_names)
# titanic_train.drop("Age_Range", axis=1, inplace=True)

In [197]:
# # 형제자매 유무에 따른 생존 결과
# print(titanic_train.groupby(["SibSp","Survived"])["Survived"].count())
# sns.barplot(x="SibSp", y="Survived", data=titanic_train)

In [198]:
# # 부모자식 관계에 따른 생존 결과
# print(titanic_train.groupby(["Parch","Survived"])["Survived"].count())
# sns.barplot(x="Parch", y="Survived", data=titanic_train)

In [199]:
# # 티켓 요금에 따른 생존 결과
# ticket_fare = ["0~19", "20~39", "40~59", "고가의 티켓"]
# titanic_train["Ticket_fare"]=titanic_train["Fare"].apply(lambda x: "0~19" if x<20 else "20~39" if x<40 else "40~59" if x<60 else "고가의 티켓")
# print(titanic_train.groupby(["Ticket_fare","Survived"])["Survived"].count())
# sns.barplot(x="Ticket_fare", y="Survived", data=titanic_train, order=ticket_fare)
# titanic_train.drop("Ticket_fare", axis=1, inplace=True)

In [200]:
# # 선실에 따른 생존 결과
# cabin = ["A", "B", "C", "D", "E", "F", "G", "N", "T"]
# print(titanic_train.groupby(["Cabin","Survived"])["Survived"].count())
# sns.barplot(x="Cabin", y="Survived", data=titanic_train, order=cabin)

In [201]:
# # 승선지에 따른 생존 결과
# print(titanic_train.groupby(["Embarked","Survived"])["Survived"].count())
# sns.barplot(x="Embarked", y="Survived", data=titanic_train)

In [202]:
# # 빈부격차에 따른 생존 
# sns.barplot(x="Pclass", y="Survived", hue="Sex", data=titanic_train)

## ML

## 유용한 함수 생성 (결측치 제거, 열 정제, 인코딩)

In [5]:
# Null 처리 함수
def fillna(df):
    df["Age"].fillna(df["Age"].mean(), inplace=True)
    df["Cabin"].fillna("N", inplace=True)
    df["Embarked"].fillna("N", inplace=True)
    df["Fare"].fillna(0, inplace=True)
    return df

# ML 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행
def format_features(df):
    df["Cabin"] = df["Cabin"].str[:1]
    features = ["Cabin","Sex","Embarked"]
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

## 데이터 정제

In [45]:
from sklearn.preprocessing import LabelEncoder

# 원본 데이터 재로딩하고, 피처 데이터 세트와 레이블 데이터 세트 추출
titanic_df = pd.read_csv("./titanic/titanic_train.csv")
y_titanic_df = titanic_df["Survived"]
x_titanic_df = titanic_df.drop("Survived", axis = 1)

x_titanic_df = transform_features(x_titanic_df)

## 다양한 모델 비교

In [94]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np


x_train, x_test, y_train, y_test = train_test_split(x_titanic_df, y_titanic_df, test_size = 0.2, random_state=11)


# 세 개 모델에 객체 생성
titanic_DT = DecisionTreeClassifier(random_state=11)
titanic_RF = RandomForestClassifier(random_state=11)
titanic_LR = LogisticRegression()


# DecisionTreeClassifier
titanic_DT.fit(x_train, y_train)
pred_dt = titanic_DT.predict(x_test)
print("DecisionTresCalssifier", np.round(accuracy_score(y_test, pred_dt), 4))

# RandomForestClassifier
titanic_RF.fit(x_train, y_train)
pred_rf = titanic_RF.predict(x_test)
print("RandomForestClassifier", np.round(accuracy_score(y_test, pred_rf), 4))

# LogisticRegression
titanic_LR.fit(x_train, y_train)
pred_lr = titanic_LR.predict(x_test)
print("LogisticRegression", np.round(accuracy_score(y_test, pred_lr), 4))

DecisionTresCalssifier 0.7877
RandomForestClassifier 0.8547
LogisticRegression 0.8492


/Users/ryleyun/opt/anaconda3/envs/nlppython/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


##### -> RandomForestClassifier가 높음. Ensemble 많이 쓰는 이유

## 의사 결정 나무 + K-Fold

In [90]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

# 의사 결정 나무 객체 생성
titanic_DT = DecisionTreeClassifier(random_state=11)

# KFold 지정
kfold = KFold(n_splits = 5)
cv_accuracy = []
n_iter = 0


# 함수 생성
def exec_kfold(clf, folds=5):
    kfold = KFold(n_splits = folds)
    scores = []

# train, test 데이터 k_fold에 따라 나누기
    for iter_count, (train_index, test_index) in enumerate(kfold.split(x_titanic_df)):

        x_train, x_test = x_titanic_df.iloc[train_index], x_titanic_df.iloc[test_index]
        y_train, y_test = y_titanic_df[train_index], y_titanic_df[test_index]


        clf.fit(x_train, y_train)
        pred = clf.predict(x_test)
        accuracy = np.round(accuracy_score(y_test, pred), 4)
        scores.append(accuracy)

        print("\n#{0} 교차 검증 정확도: {1}".format(iter_count, accuracy))
    
    mean_score = np.mean(scores)
    print("\n## 평균 검증 정확도 : ", round(mean_score, 4))
    
    
exec_kfold(titanic_DT, folds=10)


#0 교차 검증 정확도: 0.7333

#1 교차 검증 정확도: 0.7753

#2 교차 검증 정확도: 0.7416

#3 교차 검증 정확도: 0.7528

#4 교차 검증 정확도: 0.809

#5 교차 검증 정확도: 0.809

#6 교차 검증 정확도: 0.809

#7 교차 검증 정확도: 0.764

#8 교차 검증 정확도: 0.8652

#9 교차 검증 정확도: 0.8427

## 평균 검증 정확도 :  0.7902


## 의사 결정 나무 + StratifiedKfold

In [80]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

# 의사 결정 나무 객체 생성
titanic_DT = DecisionTreeClassifier(random_state=11)

# KFold 지정
skfold = StratifiedKFold(n_splits = 5)
cv_accuracy = []
n_iter = 0

# train, test 데이터 StratifiedKFold에 따라 나누기, y값 넣기
for train_index, test_index in skfold.split(x_titanic_df, y_titanic_df):
    
    x_train, x_test = x_titanic_df.iloc[train_index], x_titanic_df.iloc[test_index]
    y_train, y_test = y_titanic_df[train_index], y_titanic_df[test_index]
    
    
    titanic_DT.fit(x_train, y_train)
    pred = titanic_DT.predict(x_test)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    
    print("\n#{0} 교차 검증 정확도: {1}, 학습데이터 크기: {2}, 검증데이터 크기: {3}".format(n_iter, accuracy, train_size, test_size))
    cv_accuracy.append(accuracy)

    
print("\n## 평균 검증 정확도 : ", np.mean(cv_accuracy))


#1 교차 검증 정확도: 0.743, 학습데이터 크기: 712, 검증데이터 크기: 179

#2 교차 검증 정확도: 0.7753, 학습데이터 크기: 713, 검증데이터 크기: 178

#3 교차 검증 정확도: 0.7921, 학습데이터 크기: 713, 검증데이터 크기: 178

#4 교차 검증 정확도: 0.7865, 학습데이터 크기: 713, 검증데이터 크기: 178

#5 교차 검증 정확도: 0.8427, 학습데이터 크기: 713, 검증데이터 크기: 178

## 평균 검증 정확도 :  0.78792


## 의사 결정 나무 + cross_val_score

In [60]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate

scores = cross_val_score(titanic_DT, x_titanic_df, y_titanic_df, scoring="accuracy", cv=5)

print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.743  0.7753 0.7921 0.7865 0.8427]
평균 검증 정확도: 0.7879


## GridSearchCV()

In [110]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score

parameters = {"max_depth":[2,3,5,10], "min_samples_split":[2,3,5], "min_samples_leaf":[1,5,8]}

grid_dtree = GridSearchCV(titanic_DT, param_grid=parameters, scoring="accuracy", cv = 5)

grid_dtree.fit(x_train, y_train)

print("GridSearchCV 최적 하이퍼 파라미터: ", grid_dtree.best_params_)
print("GridSearchCV 최고 정확도: {0:.4f}".format(grid_dtree.best_score_))
best_dclf = grid_dtree.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 estimator로 예측 및 평가 수행
dpredictions = best_dclf.predict(x_test)

accuracy = accuracy_score(y_test, dpredictions)
precision = precision_score(y_test, dpredictions)
recall = recall_score(y_test, dpredictions)

print("테스트 세트에서의 DecisionTreeClassifier 정확도: {0:.4f}".format(accuracy))
print("테스트 세트에서의 DecisionTreeClassifier 정밀도: {0:.4f}".format(precision))
print("테스트 세트에서의 DecisionTreeClassifier 재현율: {0:.4f}".format(recall))

GridSearchCV 최적 하이퍼 파라미터:  {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.7992
테스트 세트에서의 DecisionTreeClassifier 정확도: 0.8715
테스트 세트에서의 DecisionTreeClassifier 정밀도: 0.8393
테스트 세트에서의 DecisionTreeClassifier 재현율: 0.7705


- 정확도(accuracy): TP+TN/TP+FP+FN+TN (얼마나 정답을 맞춰는가) 
- 정밀도(precision): TP/TP+FP (모델이 True라고 말한 것 중 정답이 몇개인가)
- 재현율(recall): TP?TP+FN (원래 값이 True인 것에 대해 모델이 얼마나 True라고 재현했는가)
- 정밀도와 재현율의 중간값(F1 Score): 2*(precision*recall)/(precision+recall)